In [1]:
# -*- coding:utf-8 -*-
"""
Description: 
    1) Use the pre handled data(image description and image features to train the caption geneartion model
    2) firstly quick choose a model to train
Author: allocator
"""

'\nDescription: \n    1) Use the pre handled data(image description and image features to train the caption geneartion model\n    2) firstly quick choose a model to train\nAuthor: allocator\n'

In [2]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalMaxPooling2D
import numpy as np
from numpy import array
from numpy import argmax
from pandas import DataFrame
from nltk.translate.bleu_score import corpus_bleu
import os
import numpy.random as rd
import json
import h5py

Using TensorFlow backend.


In [3]:
img_dir = '../data/img'
img_feature_dir = '../data/img_feature'
img_feature_file = 'image_features.h5'
clean_txt_dir = '../data/clean_txt'
clean_txt_file = 'image_descs.json'
set_category = 200
seed = 10
output_dir = '../data/res'
output_filename = 'development_dataset_id.json'

In [4]:
# compare the image feature list and the image desc list
def compare_feature_desc(feature_file, desc_file):
    """Compare the image feature and desc."""
    image_features = h5py.File(feature_file, 'r')
    image_descs = json.load(open(desc_file, 'r'))
    image_feature_keys = dict(image_features.keys())
    image_descs_keys = image_descs.keys()
    print(' current image_feature_keys')
    print(image_feature_keys)
    print(' current image_descs_keys')
    print(image_descs_keys)

In [5]:
# first seperate the data set
def seperate_dataset(img_dir, category, seed):
    """Generate the development set to quickly choose the model and configuration."""
    img_list = os.listdir(img_dir)
    # random select the train set and test set from the image list
    img_list = [item.split('.')[0] for item in img_list]
    img_size = len(img_list)
    extract_set = set()
    dataset = {}
    rd.seed(seed)
    while len(extract_set) < category:
        curr_id = rd.randint(img_size)
        curr_item = img_list[curr_id]
        if curr_item not in extract_set:
            extract_set.add(curr_item)
    print(' extract set generated and length %d ' % len(extract_set))
    set_len = int(category/2)
    dataset['train'] = list(extract_set)[:set_len]
    dataset['test'] = list(extract_set)[set_len:]
    return dataset

In [6]:
# save the dataset id
def save_dataset(filename, dataset):
    """Save the dataset."""
    file_path = os.path.join(output_dir, filename)
    json.dump(dataset, open(file_path, 'w'))

In [7]:
# load the text and each photo has one complete sentence
def load_txt(filename, dataset):
    train_set = {}
    test_set = {}
    train_id_list = dataset['train']
    test_id_list = dataset['test']
    image_descs = json.load(open(filename, 'r'))
    for item in train_id_list:
        train_set[item] = 'startseq ' + ' '.join(image_descs[item]) + ' endseq'
    for item in test_id_list:
        test_set[item] = 'startseq ' + ' '.join(image_descs[item]) + ' endseq'
    return train_set, test_set

In [8]:
# load the features about the images
def load_image_feature(filename, dataset):
    """Load the image feature about the train and test image id list."""
    image_features = h5py.File(filename, 'r')
    train_set = {}
    test_set = {}
    train_id_list = dataset['train']
    test_id_list = dataset['test']
    for item in train_id_list:
        train_set[item] = np.array(image_features[item])
    for item in test_id_list:
        test_set[item] = np.array(image_features[item])
    return train_set, test_set

In [9]:
# to encode the descriptions before training
# from words to unique integer values
# analyze the Tokenizer and fit_on_texts
def create_tokenizer(descriptions):
    """Encode the descriptions to numbers for model training."""
    tokenizer = Tokenizer()
    # each line contains the the description sentence about the image
    lines = list(descriptions.values())
    # 0 is a reserved index so the word's index starts with 1
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [10]:
# the important generate training sequences

In [11]:
# create the sequence of the images input sequences and output words for an image
def create_sequences(tokenizer, image_desc, image, max_length):
    ximage, xseqs, y = list(), list(), list()
    # encode the description with integer
    # pluse one because the index of 0 is reversed 
    # when calculate the vocab_size should add the reversed 0
    vocab_size = len(tokenizer.word_index) + 1
    sequence = tokenizer.texts_to_sequences([image_desc])[0]
    # splite current sequence to multiple x,y pairs
    for i in range(1, len(sequence)):
        # split the sequence
        input_seq, output_seq = sequence[:i], sequence[i]
        # pad input sequence to make the sequence same length
        # max length is used to limit the input sequences
        input_seq = pad_sequences([input_seq], maxlen=max_length)[0]
        # encode output make the output as a categorical list
        output_seq = to_categorical([output_seq], num_classes=vocab_size)[0]
        ximage.append(image)
        xseqs.append(input_seq)
        y.append(output_seq)
    # each picture contains seqs images output are list
    return [ximage, xseqs, y]

In [12]:
# some explain about the model
# the structure of the model and the design of the model

In [13]:
# begin to fit the model
def define_model(vocab_size, max_length):
    """Define the model."""
    # image feature map
    inputs1 = Input(shape=(7, 7, 512))
    fe1 = GlobalMaxPooling2D()(inputs1)
    fe2 = Dense(128, activation='relu')(fe1)
    fe3 = RepeatVector(max_length)(fe2)
    # embedding
    inputs2 = Input(shape=(max_length,))
    emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
    emb3 = LSTM(256, return_sequences=True)(emb2)
    emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
    # merge inputs image descs sequence and image features
    merged = concatenate([fe3, emb4])
    # language model (decoder)
    lm2 = LSTM(500)(merged)
    lm3 = Dense(500, activation='relu')(lm2)
    outputs = Dense(vocab_size, activation='softmax')(lm3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    # plot_model(model, show_shapes=True, to_file='plot.png')
    return model

In [14]:
# data generator, intended to be used in a call to model.fit_generator()
# it is an important generator
def data_generator(descriptions, features, tokenizer, max_length, n_step):
    # loop until we finish training
    # n_step means the procedure of each epoch
    while 1:
        # loop over photo identifiers in the dataset
        keys = list(descriptions.keys())
        for i in range(0, len(keys), n_step):
            Ximages, XSeq, y = list(), list(),list()
            # maybe the last batch is less than the normal batch
            for j in range(i, min(len(keys), i+n_step)):
                image_id = keys[j]
                # retrieve photo feature input
                image = features[image_id][0]
                # retrieve text input
                desc = descriptions[image_id]
                # generate input-output pairs
                in_img, in_seq, out_word = create_sequences(tokenizer, desc, image, max_length)
                for k in range(len(in_img)):
                    Ximages.append(in_img[k])
                    XSeq.append(in_seq[k])
                    y.append(out_word[k])
            # yield this batch of samples to the model
            # normal result is array with [x_input,y_output] each 
            yield [[array(Ximages), array(XSeq)], array(y)]

In [15]:
# map an integer to a word
# from the tokenizer id to the word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [16]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

In [17]:
# evaluate the skill of the model
# use the trained model and return the bleu value
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    # step over the whole set
    for key, desc in descriptions.items():
        # generate description
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        # store actual and predicted
        actual.append([desc.split()])
        predicted.append(yhat.split())
    # calculate BLEU score
    bleu = corpus_bleu(actual, predicted)
    return bleu

In [18]:
dataset = seperate_dataset(img_dir, set_category, seed)
# save_dataset(output_filename, dataset)

 extract set generated and length 200 


In [19]:
# compare_feature_desc(os.path.join(img_feature_dir, img_feature_file), os.path.join(clean_txt_dir, clean_txt_file))
# train_desc and test_desc have the complete sentence of the photo description
train_desc, test_desc = load_txt(os.path.join(clean_txt_dir, clean_txt_file), dataset)
# the shape of the train_img is (1,7,7,512) ndarray
train_img, test_img = load_image_feature(os.path.join(img_feature_dir, img_feature_file), dataset)

In [20]:
# validate the train set
# print(type(train_img))
# print(list(train_img.keys()))
# print(type(train_img['img_5519']))
# print(train_img['img_5519'].shape)

In [ ]:
# some information about the trainging and test data set
# print(' train_desc length %d test_desc length %d ' % (len(train_desc), len(test_desc)))
# print(' train_img length %d test_img length %d ' % (len(train_img), len(test_img)))

In [ ]:
# begin to train the model
# get the tokenizer
# just use the trainig desc to get the tokenizer
# and for the test desc still use the trainig tokenizer
tokenizer = create_tokenizer(train_desc)
vocab_size = len(tokenizer.word_index) + 1
# get the maximum length of the descriptions
max_length = max([len(item.split()) for item in list(train_desc.values())])
print(' The maximum length of the description is %d ' % max_length)

# define the experiment
model_name = 'basiccaptionmodel'
verbose = 2
# set the iterate times
n_epochs = 50
# set the picture update number
n_photos_per_update = 2
# calculate the batches per epoch
n_batches_per_epoch = int(len(dataset['train']) / n_photos_per_update)
# set the repeat number there is no k fold cross validation
# in this training procedure there is no cross validation
# in this part normally the repeats will be at list 30 times but consider the time
# just 3 times for the pre test
n_repeats = 3

# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
    # define the model
    model = define_model(vocab_size, max_length)
    # fit model
    model.fit_generator(data_generator(train_desc, train_img, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
    # evaluate model on training data
    train_score = evaluate_model(model, train_desc, train_img, tokenizer, max_length)
    # evaluate the model on test data
    test_score = evaluate_model(model, test_desc, test_img, tokenizer, max_length)
    # store
    train_results.append(train_score)
    test_results.append(test_score)
    print('---%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(' current training result ')
print(df.describe())
df.to_csv(os.path.join(output_dir, model_name+'.csv'), index=False)

 The maximum length of the description is 70 
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 512)          0           input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 70, 50)       42850       input_2[0][0]                    
_______________________________________________________________

D:\programfiles\anaconda3\envs\tensorflowenv\lib\site-packages\nltk\translate\bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


---1: train=0.068699 test=0.067455
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 70)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_2 (GlobalM (None, 512)          0           input_3[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 70, 50)       42850       input_4[0][0]                    
__________________________________________________________________________

Epoch 1/50
 - 178s - loss: 6.0147 - acc: 0.0460
Epoch 2/50
 - 174s - loss: 5.6707 - acc: 0.0470
Epoch 3/50
 - 175s - loss: 5.6263 - acc: 0.0435
Epoch 4/50
 - 177s - loss: 5.5943 - acc: 0.0486
Epoch 5/50
 - 185s - loss: 5.5437 - acc: 0.0483
Epoch 6/50
 - 182s - loss: 5.4842 - acc: 0.0519
Epoch 7/50
 - 180s - loss: 5.4466 - acc: 0.0521
Epoch 8/50
 - 175s - loss: 5.4127 - acc: 0.0528
Epoch 9/50
 - 175s - loss: 5.3820 - acc: 0.0541
Epoch 10/50
 - 177s - loss: 5.3610 - acc: 0.0538
Epoch 11/50
 - 175s - loss: 5.3475 - acc: 0.0515
Epoch 12/50
 - 180s - loss: 5.3700 - acc: 0.0468
Epoch 13/50
 - 182s - loss: 5.3442 - acc: 0.0502
Epoch 14/50
 - 177s - loss: 5.3194 - acc: 0.0538
Epoch 15/50
 - 178s - loss: 5.3209 - acc: 0.0530
Epoch 16/50
 - 188s - loss: 5.3072 - acc: 0.0487
Epoch 17/50
 - 174s - loss: 5.2941 - acc: 0.0467
Epoch 18/50
 - 171s - loss: 5.2825 - acc: 0.0499
Epoch 19/50
 - 175s - loss: 5.2703 - acc: 0.0447
Epoch 20/50
 - 173s - loss: 5.2703 - acc: 0.0500
Epoch 21/50
 - 174s - loss: 5